In [15]:
!pip install tensorflow
!pip install tflearn
import tflearn
import tensorflow as tf

In [16]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stem = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
import numpy as np
import pandas as pd
import random
import json

In [18]:
with open('intents2.json') as json_data:
    intents = json.load(json_data)

In [19]:
words = []
classes = []
docs = []
ignore = ['?']


for intent in intents['intents']:

    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)
        docs.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [20]:
words = [stem.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(docs), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

47 documents
11 classes ['contact', 'education', 'experience', 'goodbye', 'greeting', 'how', 'location', 'real', 'skills-libraries', 'thanks', 'who']
61 unique stemmed words ["'s", 'a', 'about', 'ai', 'alright', 'anyon', 'ar', 'background', 'bit', 'bye', 'can', 'colleg', 'contact', 'day', 'dhruv', 'did', 'do', 'doe', 'educ', 'expery', 'go', 'good', 'goodby', 'hav', 'he', 'hello', 'help', 'hi', 'how', 'i', 'is', 'know', 'langu', 'lat', 'libr', 'littl', 'loc', 'mak', 'me', 'poss', 'program', 'reach', 'real', 'remot', 'school', 'see', 'skil', 'tel', 'thank', 'that', 'ther', 'thi', 'to', 'what', 'wher', 'who', 'wil', 'with', 'work', 'yo', 'you']


In [21]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in docs:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stem.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [22]:
tf.compat.v1.reset_default_graph()

layer = tflearn.input_data(shape=[None, len(train_x[0])])
layer = tflearn.fully_connected(layer, 15)
layer = tflearn.fully_connected(layer, 5)
layer = tflearn.fully_connected(layer, 10)
layer = tflearn.fully_connected(layer, len(train_y[0]), activation='softmax')
layer = tflearn.regression(layer)

model = tflearn.DNN(layer, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=10, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.55872 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.55872 - acc: 0.9756 -- iter: 40/47
Training Step: 5000  | total loss: 0.45470 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.45470 - acc: 0.9780 -- iter: 47/47
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [23]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [24]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [25]:
with open('intents2.json') as json_data:
    intents = json.load(json_data)

In [26]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [27]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stem.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bagger(sentence, words):
    sentence_words = clean_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1

    return(np.array(bag))

In [28]:
ERROR_MIN = 0.30
def return_class(sentence):
    result = model.predict([bagger(sentence, words)])[0]
    result = [[i,r] for i,r in enumerate(result) if r>ERROR_MIN]

    result.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in result:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def return_response(sentence):
    result = return_class(sentence)
    if result:
        while result:
            for i in intents['intents']:
                if i['tag'] == result[0][0]:
                    return print(random.choice(i['responses']))

            result.pop(0)

In [35]:
return_class("Hello, greetings!")

[('greeting', 0.98475295)]

In [36]:
return_response("Hello, greetings!")

Hello, I am a chatbot. I answer questions about Dhruv.


In [39]:
return_class("Testing something that doesn't necessarily fall into one clear-cut category")
# interesting categorization with a surprisingly high confidence, looks like something to work on..

[('goodbye', 0.7467977)]

In [42]:
return_response("Tell me a little about yourself, will you?")
# note that the input is not something the model has been trained on

Dhruv is an AI focused programmer. You can read more about him at https://dhruvkulkarni.info/#about


In [41]:
return_response("Where are you from, Dhruv?")
# note that my input is not something that the model has been trained on 

I am currently based in Los Angeles, but I am open to working remotely.


In [43]:
return_response("What is your background?")

I work with artificial intelligence and much more. Read more at https://dhruvkulkarni.info/#skills and https://dhruvkulkarni.info/#areas


In [44]:
return_response("What kind of experience do you have?")

I worked as a Data Science and SWE intern at Photon Commerce along with a couple other things, but you can find out more here: https://dhruvkulkarni.info/#experience


In [45]:
return_response("How can I reach you or contact you?")

You can reach me at DhruvK0003@gmail.com and I will answer your email as soon as I get it.


In [46]:
return_response("Thanks, goodbye!")

Any time!
